# Python Exercise 14: dealing with databases - Homework 3


### This exercise is part you the homework you need to submit for your final grade

* Please try to answer all the questions in this exercise and send your solution to me via email before the end of May 30th 2017.
* This homework is worth 15% of your final grade
* Please prepare and submit your Exercise according to the instructions below. **I will deduce points** if your exercise is not submitted according to the instructions or is submitted late.
* For each question you should provide source to answer it and summarize your findings (in Chinese or English, in any case I will not grade your English as long as I can understand the meaning).

### Instructions for submission
* Enter your name, major and your student number in the field below (in Chinese). Double click on the field to edit it.

**Name:**  项闰冶 **major** 计算机科学与技术 **student number** 140104200230      

* Change the name of the notebook to the following format
```
Python HW 3 <yourname> <your student no>
```
(you can change the name in the menu below `File->Rename`. So, if you name is 马小龙 and your student number is 2018101111000111, you should change the name of the notebook to `Python HW 3 马小龙 2018101111000111`

* Export the notebook both as a HTML file and as notebook (choose `File->Download as` from the menu)
* Send both files to me via email to 2485707749@qq.com before the end of Tuesday, May 30th 2017
* your email should mention both your name and your student number in the subject
* to repeat, your name and student number should appear in three different places: in the text above, in the filename, and in the subject of your email!


**I will deduce points if your exercise is not submitted according to the instructions or is submitted late.**



### Installing sqlite3

You first should install the sqlite3 database: download the zip files `sqlite-dll-win32-x86-3190000.zip` and `sqlite-tools-win32-x86-3190000.zip` (or the corresponding files if you are using another operating system) from the website  http://sqlite.org/download.html (or from our QQ group) and unpack them. <ake sure you place the `.dll` file from the first zip in the same directory as your Python notebook.

If you are **not** using the Anaconda distribution you might also need to install the 
 ``sqlite`` database package:  Enter the following command inside a command line:

```
apt-get install sqlite3
```

### Problem 14-1:

Open a connection the a database and write a program which creates tables to store products, customers, and orders. Each product has an ID, a name, a description, and a price. In one order a customer can only order a single product, but it is possible to order multiple copies of the same product in one order. Finally, a customer has a customer ID, a name, and a telephone number.

Add at least three products, four customers, and five orders to the database. This data can be hard-coded, no need to use user-input.


In [8]:
import sqlite3
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

In [20]:
#下面三个分别为创建三张表
sql1 = '''CREATE TABLE  Products(
    productID integer,
    productName text, 
    description text, 
    price integer
)'''
cursor.execute(sql1)

In [21]:
sql2 = '''CREATE TABLE Customers (
    customerID integer,
    customerName text, 
    telephoneNumber text
)'''
cursor.execute(sql2)

In [23]:
sql3 = '''CREATE TABLE Orders (
    productID integer,
    customerID integer,
    amount integer
)'''
cursor.execute(sql3)

In [19]:
conn.execute('drop table Customers')
conn.execute('drop table Products')
conn.execute('drop table Orders')

In [24]:
#下面三个为创建数据
cursor.execute("insert into Products values (?, ?, ?,?)", 
               (1,"book", "This is a book", 38))
cursor.execute("insert into Products values (?, ?, ?,?)", 
               (2,"pen", "This is a pen", 3))
cursor.execute("insert into Products values (?, ?, ?,?)", 
               (3,"key", "This is a key", 10))

In [25]:
cursor.execute("insert into Customers values (?,?,?)", 
               (1,"A", "17816853681"))
cursor.execute("insert into Customers values (?, ?,?)", 
               (2,"B", "17816853682"))
cursor.execute("insert into Customers values (?, ?,?)", 
               (3,"C", "17816853683"))
cursor.execute("insert into Customers values (?, ?,?)", 
               (4,"D", "17816853684"))

In [26]:
cursor.execute("insert into Orders values (?, ?,?)", 
               (1,1, 10))
cursor.execute("insert into Orders values (?, ?,?)", 
               (2,2, 1))
cursor.execute("insert into Orders values (?, ?,?)", 
               (3,3, 4))
cursor.execute("insert into Orders values (?, ?,?)", 
               (1,3, 7))
cursor.execute("insert into Orders values (?, ?,?)", 
               (1,2, 5))

### Problem 14-2:

Write code to print out the information of all customers, all orders, and all products. Each order should contain the total amount of the order. For each customer print out how many orders he placed and how much money he spend. For each product, print how often it was sold and to how many different customers it was sold.

In [31]:
# order，利用where语句连接三张表
sql='''
select Orders.customerID,customerName,Orders.productID,productName,amount,amount*price
from Orders,Customers,Products
where Orders.customerID=Customers.customerID and Orders.productID=Products.productID
'''
for row in cursor.execute(sql):
    print(row)

(1, 'A', 1, 'book', 10, 380)
(2, 'B', 2, 'pen', 1, 3)
(3, 'C', 3, 'key', 4, 40)
(3, 'C', 1, 'book', 7, 266)
(2, 'B', 1, 'book', 5, 190)


In [39]:
#customer，利用left join 连接Customers和Orders表，用where连接新生成的表和Products
sql='''
select Customers.customerID,customerName,count(Orders.productID),sum(Orders.amount*price)
from Customers left join Orders on Customers.customerID=Orders.customerID,Products
where Orders.productID=Products.productID
group by Customers.customerID
'''
for row in cursor.execute(sql):
    print(row)

(1, 'A', 1, 380)
(2, 'B', 2, 193)
(3, 'C', 2, 306)


In [41]:
# 利用left join连接 Products和Orders
sql='''select  p.productID,productName,description,price,count(customerID)
from Products as p left join Orders as o on p.productID=o.productID
group by p.productID'''
for row in cursor.execute(sql):
    print(row)

(1, 'book', 'This is a book', 38, 3)
(2, 'pen', 'This is a pen', 3, 1)
(3, 'key', 'This is a key', 10, 1)
